In [1]:
import os, shutil
from time import sleep
import random 

from ai2thor.server import Event
from problem import Problem
from ai2thor.controller import Controller
from tasks import ClearDishes, SliceTomato, MakeToast, ServeCoffee, CookEgg, SetTable
import scene_info as si
from pddl2scene import PlanHandler
from scene2pddl import SceneHandler

In [2]:
kitchens = [f"FloorPlan{i}" for i in range(1, 31)]
rand_scene = random.choice(kitchens)

In [3]:
controller = Controller(
    agentMode="default",
    visibilityDistance = si.visibilityDistance,
    scene=rand_scene,

    # step sizes
    gridSize=0.25,
    snapToGrid=True,
    rotateStepDegrees=90,

    # image modalities
    renderDepthImage=False,
    renderInstanceSegmentation=False,

    branch="main",


    #camera properties
    width=384,
    height=216,
    fieldOfView=120
)

In [4]:
tasks = [ServeCoffee(),
         CookEgg()]

In [5]:
domain_filename = "pddl/robochef.pddl"
problems_path = "pddl/problems"
planner_path = "planners/LPG-td-1.4/lpg-td"
max_actions = 1000

In [6]:
##clean old data
if os.path.exists(problems_path):
    shutil.rmtree(problems_path)
os.mkdir(problems_path)

In [7]:
#model the problem
problem = Problem(controller, tasks)


In [8]:
#generate a PDDL problem file
problem_filename = SceneHandler(problem).ToPDDL(problems_path)


File pddl/problems/problem_18948.pddl written successfully


In [11]:
#use planner to get a solution
plan_filepath, l_from, l_to = SceneHandler.Plan(planner_path, domain_filename, problem_filename)
plan_handler = PlanHandler(plan_filepath, problem, l_from, l_to)

In [17]:
#execute actions
for i in range(max_actions):
    action = plan_handler.parse_line()
    if action is not None:
        event = action.execute()
        if event is not None:
            Problem.PrintLastActionStatus(event, action.args)
        sleep(0.5)
    else:
        problem.controller.step('Pass')
        input("Mission finished! Press Enter to continue...")
        break

SliceObject--['egg1', 'pan1']--successful
CookObject--['egg1', 'pan1', 'stoveburner1']--unsuccesful
Error:  object not found: Egg|+00.71|+00.94|+00.94|EggCracked_0
PickupObject--['egg1']--unsuccesful
Error:  NullReferenceException: Target object not found within the specified visibility.
Done--['egg1', 'plate1']--successful
TeleportFull--['plate1']--successful
PutObject--['egg1', 'plate1']--unsuccesful
Error:  No valid positions to place object found
PickupObject--['plate1']--unsuccesful
Error:  InvalidOperationException: Agent hand has something in it already! Can't pick up anything else
Done--['plate1', 'countertop1']--successful
TeleportFull--['countertop1']--successful
PutObject--['plate1', 'countertop1']--successful
Done--['countertop1', 'mug1']--successful
TeleportFull--['mug1']--successful
PickupObject--['mug1']--successful
Done--['mug1', 'coffeemachine1']--successful
TeleportFull--['coffeemachine1']--successful
PutObject--['mug1', 'coffeemachine1']--successful
ToggleObjectOn--[

KeyboardInterrupt: Interrupted by user